Animation Code snippet for GPS Data

In [ ]:
# pip install folium pandas openpyxl

In [ ]:
pip install xlrd

In [2]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# =====================
# Load your dataset
# =====================
# df = pd.read_excel("03-06-25_Car_Navicalone.xls")
df = pd.read_excel("Dec6_Kotturpuram_Naviconly\Dec6_Kotturpuram_naviconly_10.43 am trip.xls")


# =====================
# Check column data types
# =====================
print(df.dtypes)  # Debug: see what types you have

# =====================
# Combine Date & Time into a single datetime column
# =====================
# Convert to string first, then combine
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%d-%m-%Y")
df["Time"] = df["Time"].astype(str)

df["Datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d-%m-%Y %H:%M:%S",
    errors="coerce"
)

# Drop rows with invalid datetime
df = df.dropna(subset=["Latitude", "Longitude", "Datetime"]).reset_index(drop=True)

# =====================
# Create Folium Map
# =====================
m = folium.Map(
    location=[df["Latitude"].mean(), df["Longitude"].mean()],
    zoom_start=18
)

# Start marker
folium.Marker(
    [df.iloc[0]["Latitude"], df.iloc[0]["Longitude"]],
    popup=f"Start: {df.iloc[0]['Datetime']}",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# End marker
folium.Marker(
    [df.iloc[-1]["Latitude"], df.iloc[-1]["Longitude"]],
    popup=f"End: {df.iloc[-1]['Datetime']}",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# =====================
# Build GeoJSON Features
# =====================
features = []
for _, row in df.iterrows():
    if pd.isna(row["Latitude"]) or pd.isna(row["Longitude"]):
        continue
    
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]],
        },
        "properties": {
            "time": row["Datetime"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"Time: {row['Datetime']}",
            "style": {"color": "blue"},
            "icon": "circle",
            "iconstyle": {
                "fillColor": "orange",
                "fillOpacity": 1,
                "stroke": "true",
                "radius": 8
            }
        }
    })

timestamped_geojson = {
    "type": "FeatureCollection",
    "features": features
}

# =====================
# Add animated layer
# =====================
TimestampedGeoJson(
    timestamped_geojson,
    period="PT1S",
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=2,
    loop_button=True,
    date_options="HH:mm:ss",
    time_slider_drag_update=True
).add_to(m)

# =====================
# Export to HTML
# =====================
m.save("Dec6_Kotturpuram_naviconly_10.43 am trip.xls.html")
print("✅ Animation exported to 03-06-25_Car_Navicalone.html")

Datetimecombined    datetime64[ns]
Latitude                   float64
Longitude                  float64
Date                datetime64[ns]
Time                        object
dtype: object
✅ Animation exported to 03-06-25_Car_Navicalone.html


DGPS Data Animation using Folium and Pandas, using Object ID as similar as timestamp

In [1]:
import pandas as pd
import folium
from folium.plugins import TimestampedGeoJson
from pyproj import Transformer

# =====================
# Load data
# =====================
df = pd.read_excel("Path wise sorted DGPS Kotturpuram.xlsx")  # or read_csv

# =====================
# Convert UTM → Lat/Lon
# =====================
# ⚠️ Change EPSG if needed
# Example: EPSG:32644 = UTM zone 44N (Tamil Nadu region)
transformer = Transformer.from_crs("EPSG:32644", "EPSG:4326", always_xy=True)

df["Longitude"], df["Latitude"] = transformer.transform(
    df["Easting"].values,
    df["Northing"].values
)

# =====================
# Create synthetic time using ID
# =====================
start_time = pd.Timestamp("2025-01-01 10:00:00")

# df = df.sort_values("ID").reset_index(drop=True)
df["Datetime"] = start_time + pd.to_timedelta(df.index, unit="s")

# =====================
# Create Folium map
# =====================
m = folium.Map(
    location=[df["Latitude"].mean(), df["Longitude"].mean()],
    zoom_start=18
)

# Start marker
folium.Marker(
    [df.iloc[0]["Latitude"], df.iloc[0]["Longitude"]],
    popup=f"Start | ID: {df.iloc[0]['ID']} | {df.iloc[0]['DES']}",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# End marker
folium.Marker(
    [df.iloc[-1]["Latitude"], df.iloc[-1]["Longitude"]],
    popup=f"End | ID: {df.iloc[-1]['ID']} | {df.iloc[-1]['DES']}",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# =====================
# Build GeoJSON features
# =====================
features = []

for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]],
        },
        "properties": {
            "time": row["Datetime"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": (
                f"ID: {row['ID']}<br>"
                f"DES: {row['DES']}<br>"
                f"Elevation: {row['Elevation']} m"
            ),
            "icon": "circle",
            "iconstyle": {
                "fillColor": "orange",
                "fillOpacity": 1,
                "stroke": "true",
                "radius": 7
            }
        }
    })

timestamped_geojson = {
    "type": "FeatureCollection",
    "features": features
}

# =====================
# Add animated layer
# =====================
TimestampedGeoJson(
    timestamped_geojson,
    period="PT1S",          # 1 second per ID
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=5,
    loop_button=True,
    date_options="HH:mm:ss",
    time_slider_drag_update=True
).add_to(m)

# =====================
# Save output
# =====================
m.save("Path wise sorted DGPS_ID_based_animation.html")
print("✅ Animation exported to Path wise sorted DGPS_ID_based_animation.html")

✅ Animation exported to Path wise sorted DGPS_ID_based_animation.html


Above code doesn't work if there are NaN, below is the corrected code

In [3]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# =====================
# Load your dataset
# =====================
df = pd.read_csv("Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.csv")  # <-- replace with your file name

# =====================
# Combine Date & Time into a single datetime column
# =====================
# Example Date format: 04-06-2025  Time format: 18:49:50
# Convert to string first, then combine
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%d-%m-%Y")
df["Time"] = df["Time"].astype(str)

df["Datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d-%m-%Y %H:%M:%S",
    errors="coerce"
)

# =====================
# Drop rows missing coordinates or invalid datetime
# =====================
df = df.dropna(subset=["Latitude", "Longitude", "Datetime"]).reset_index(drop=True)

# =====================
# Create Folium Map
# =====================
m = folium.Map(
    location=[df["Latitude"].mean(), df["Longitude"].mean()],
    zoom_start=18
)

# Start marker
folium.Marker(
    [df.iloc[0]["Latitude"], df.iloc[0]["Longitude"]],
    popup=f"Start: {df.iloc[0]['Datetime']}",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# End marker
folium.Marker(
    [df.iloc[-1]["Latitude"], df.iloc[-1]["Longitude"]],
    popup=f"End: {df.iloc[-1]['Datetime']}",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# =====================
# Build GeoJSON Features (skip missing coords)
# =====================
features = []
for _, row in df.iterrows():
    # Skip if latitude or longitude is missing or invalid
    if pd.isna(row["Latitude"]) or pd.isna(row["Longitude"]):
        continue
    
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["Longitude"], row["Latitude"]],
        },
        "properties": {
            "time": row["Datetime"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"Time: {row['Datetime']}",
            "style": {"color": "blue"},
            "icon": "circle",
            "iconstyle": {
                "fillColor": "orange",
                "fillOpacity": 1,
                "stroke": "true",
                "radius": 8
            }
        }
    })

timestamped_geojson = {
    "type": "FeatureCollection",
    "features": features
}

# =====================
# Add animated layer
# =====================
TimestampedGeoJson(
    timestamped_geojson,
    period="PT1S",               # 1-second interval
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=2,
    loop_button=True,
    date_options="HH:mm:ss",     # show only time
    time_slider_drag_update=True
).add_to(m)

# =====================
# Export to HTML
# =====================
m.save("Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.html")
print("✅ Animation exported to Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.html")

✅ Animation exported to Dec6_Kotturpuram_naviconly_12.53 pm trip_HMM.html


below is the code that works for animation of GPS coordinates using Folium and Pandas, with nans, names as matched_latitude and matched_longitude

In [1]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# =====================
# Load your dataset
# =====================
df = pd.read_csv("Dec6_Kotturpuram_naviconly_10.43 am trip_HMM.csv")  # <-- replace with your file name

# =====================
# Combine Date & Time into a single datetime column
# =====================
# Example Date format: 04-06-2025  Time format: 18:49:50
df["Datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d-%m-%Y %H:%M:%S",
    errors="coerce"
)

# =====================
# Drop rows missing coordinates or invalid datetime
# =====================
# here we need to animate the matched_latitude, matched_longitude
# so lets change the column names accordingly
df = df.dropna(subset=["matched_latitude", "matched_longitude", "Datetime"]).reset_index(drop=True)

# =====================
# Create Folium Map
# =====================
m = folium.Map(
    location=[df["matched_latitude"].mean(), df["matched_longitude"].mean()],
    zoom_start=18
)

# Start marker
folium.Marker(
    [df.iloc[0]["matched_latitude"], df.iloc[0]["matched_longitude"]],
    popup=f"Start: {df.iloc[0]['Datetime']}",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# End marker
folium.Marker(
    [df.iloc[-1]["matched_latitude"], df.iloc[-1]["matched_longitude"]],
    popup=f"End: {df.iloc[-1]['Datetime']}",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# =====================
# Build GeoJSON Features (skip missing coords)
# =====================
features = []
for _, row in df.iterrows():
    # Skip if latitude or longitude is missing or invalid
    if pd.isna(row["matched_latitude"]) or pd.isna(row["matched_longitude"]):
        continue
    
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["matched_longitude"], row["matched_latitude"]],
        },
        "properties": {
            "time": row["Datetime"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"Time: {row['Datetime']}",
            "style": {"color": "blue"},
            "icon": "circle",
            "iconstyle": {
                "fillColor": "orange",
                "fillOpacity": 1,
                "stroke": "true",
                "radius": 8
            }
        }
    })

timestamped_geojson = {
    "type": "FeatureCollection",
    "features": features
}

# =====================
# Add animated layer
# =====================
TimestampedGeoJson(
    timestamped_geojson,
    period="PT1S",               # 1-second interval
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=2,
    loop_button=True,
    date_options="HH:mm:ss",     # show only time
    time_slider_drag_update=True
).add_to(m)

# =====================
# Export to HTML
# =====================
# the final output should be (input file name) + "_animation.html"
m.save("Dec6_Kotturpuram_naviconly_10.43 am trip_HMM_animation.html")
print("✅ Animation exported to Dec6_Kotturpuram_naviconly_10.43 am trip_HMM_animation.html")

ValueError: Location values cannot contain NaNs.

below is the code for Adv hmm outpt format

In [5]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# =====================
# 1. Load the NEW dataset
# =====================
# We use the file generated by the Advanced HMM code
INPUT_CSV = "5) 17-07-25_Car_NavicplusGPS_Trip1_AdvHMM.csv"
df = pd.read_csv(INPUT_CSV)

# =====================
# 2. Prepare Datetime
# =====================
# The new CSV already has a 'datetime' column, so we just parse it.
df["Datetime"] = pd.to_datetime(df["datetime"])

# =====================
# 3. Clean Data
# =====================
# The new code uses 'matched_lat' and 'matched_lon'
df = df.dropna(subset=["matched_lat", "matched_lon", "Datetime"]).reset_index(drop=True)

# =====================
# 4. Create Folium Map
# =====================
m = folium.Map(
    location=[df["matched_lat"].mean(), df["matched_lon"].mean()],
    zoom_start=18
)

# Optional: Add the full path as a blue line so you can see the road geometry
path_coords = list(zip(df["matched_lat"], df["matched_lon"]))
folium.PolyLine(path_coords, color="blue", weight=4, opacity=0.5).add_to(m)

# Start marker
folium.Marker(
    [df.iloc[0]["matched_lat"], df.iloc[0]["matched_lon"]],
    popup=f"Start: {df.iloc[0]['Datetime']}",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

# End marker
folium.Marker(
    [df.iloc[-1]["matched_lat"], df.iloc[-1]["matched_lon"]],
    popup=f"End: {df.iloc[-1]['Datetime']}",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# =====================
# 5. Build GeoJSON Features (Animation)
# =====================
features = []
for _, row in df.iterrows():
    
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            # GeoJSON requires [Longitude, Latitude] order
            "coordinates": [row["matched_lon"], row["matched_lat"]],
        },
        "properties": {
            "time": row["Datetime"].strftime("%Y-%m-%dT%H:%M:%S"),
            "popup": f"Time: {row['Datetime']}",
            "style": {"color": "blue"},
            "icon": "circle",
            "iconstyle": {
                "fillColor": "orange",
                "fillOpacity": 1,
                "stroke": "true",
                "radius": 6 # Slightly smaller for cleaner look
            }
        }
    })

timestamped_geojson = {
    "type": "FeatureCollection",
    "features": features
}

# =====================
# 6. Add animated layer
# =====================
TimestampedGeoJson(
    timestamped_geojson,
    period="PT1S",               # 1-second interval
    add_last_point=False,        # False prevents leaving a trail of dots (cleaner)
    auto_play=True,
    loop=False,
    max_speed=5,
    loop_button=True,
    date_options="HH:mm:ss",     
    time_slider_drag_update=True
).add_to(m)

# =====================
# 7. Export
# =====================
output_html = "5) 17-07-25_Car_NavicplusGPS_Trip1_AdvHMM_animation.html"
m.save(output_html)
print(f"✅ Animation exported to {output_html}")

✅ Animation exported to 5) 17-07-25_Car_NavicplusGPS_Trip1_AdvHMM_animation.html
